# Creating a Transition Matrix to predict opponent's next move

We will create a simple markov chain with 3 states and a 3x3 transition matrix. We initialize it with uniform probabilities but then learn
the transition matrix from the data to better predict the next move of the opponent.


In [ ]:
!pip install 'kaggle-environments>=0.1.6'

In [ ]:
%%writefile submission.py

import numpy as np
import pandas as pd
import random

T = np.zeros((3, 3))
P = np.zeros((3, 3))

# a1 is the action of the opponent 1 step ago
# a2 is the action of the opponent 2 steps ago
a1, a2 = None, None

def transition_agent(observation, configuration):
    global T, P, a1, a2
    if observation.step > 1:
        a1 = observation.lastOpponentAction
        T[a2, a1] += 1
        P = np.divide(T, np.maximum(1, T.sum(axis=1)).reshape(-1, 1))
        a2 = a1
        if np.sum(P[a1, :]) == 1:
            return int((np.random.choice(
                [0, 1, 2],
                p=P[a1, :]
            ) + 1) % 3)
        else:
            return int(np.random.randint(3))
    else:
        if observation.step == 1:
            a2 = observation.lastOpponentAction
        return int(np.random.randint(3))

In [ ]:
%%writefile random.py
import numpy as np
def random_agent(observation, configuration):
    return int(np.random.randint(3))

In [ ]:
%%writefile mysubmission.py
from numpy import random
import math

last_react_action = None
def get_score(left_move, right_move):
    # This method exists in this file so it can be consumed from rps.py and agents.py without a circular dependency
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

def random_agent(observation, configuration):
    global last_react_action
    if observation.step == 0:
        my_list = [0] * 34 + [1] * 33 + [2] * 33
        last_react_action = int(random.choice(my_list))
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

In [ ]:
from kaggle_environments import evaluate, make, utils
env = make("rps", debug=True)
env.render()

In [ ]:
# env.reset()
# # Play as the first agent against default "random" agent.
# env.run(["submission.py", "random.py"])
# env.render(mode="ipython", width=500, height=450)

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run(["mysubmission.py", "submission.py"])
env.render(mode="ipython", width=500, height=450)